<a href="https://colab.research.google.com/github/ananyarao23/EmoSense-TweetSentimentDetection/blob/main/week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/WiDS-2023/week1_df1.csv')
df

,Unnamed: 0,Sentiment,SentimentText,SentimentText_stopword,TokenizedText,StemmedText,LemmatizedText,Text_Subjectivity,Text_Polarity,Text_Analysis
0,0,0,is so sad for my apl friend,sad apl friend,"['sad', 'apl', 'friend']",sad apl friend,sad apl friend,1.000000,-0.500000,Negative
1,1,0,i missed the new moon trailer,missed new moon trailer,"['missed', 'new', 'moon', 'trailer']",miss new moon trailer,missed new moon trailer,0.454545,0.136364,Positive
2,2,1,omg its already 7 30 o,omg already 7 30,"['omg', 'already', '7', '30']",omg alreadi 7 30,omg already 7 30,0.000000,0.000000,Neutral
3,3,0,omgaga im sooo im gunna cry i ve been at th...,omgaga im sooo im gunna cry dentist since 11 s...,"['omgaga', 'im', 'sooo', 'im', 'gunna', 'cry',...",omgaga im sooo im gunna cri dentist sinc 11 su...,omgaga im sooo im gunna cry dentist since 11 s...,0.000000,0.000000,Neutral
4,4,0,i think mi bf is cheating on me t t,think mi bf cheating,"['think', 'mi', 'bf', 'cheating']",think mi bf cheat,think mi bf cheating,0.000000,0.000000,Neutral
...,...,...,...,...,...,...,...,...,...,...
1578607,1578607,1,zzzzzz finally night tweeters,zzzzzz finally night tweeters,"['zzzzzz', 'finally', 'night', 'tweeters']",zzzzzz final night tweeter,zzzzzz finally night tweeter,1.000000,0.000000,Neutral
1578608,1578608,1,zzzzzzz sleep well people,zzzzzzz sleep well people,"['zzzzzzz', 'sleep', 'well', 'people']",zzzzzzz sleep well peopl,zzzzzzz sleep well people,0.000000,0.000000,Neutral
1578609,1578609,0,zzzzzzzzzz wait no i have homework,zzzzzzzzzz wait homework,"['zzzzzzzzzz', 'wait', 'homework']",zzzzzzzzzz wait homework,zzzzzzzzzz wait homework,0.000000,0.000000,Neutral
1578610,1578610,0,zzzzzzzzzzzzz meh what am i doing up again,zzzzzzzzzzzzz meh,"['zzzzzzzzzzzzz', 'meh']",zzzzzzzzzzzzz meh,zzzzzzzzzzzzz meh,0.000000,0.000000,Neutral


In [ ]:
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(df['SentimentText'].values)

In [ ]:
X = tokenizer.texts_to_sequences(df['SentimentText'].values)
X = pad_sequences(X)

In [ ]:
X

array([[  0,   0,   0, ...,  11,   5, 254],
       [  0,   0,   0, ..., 262,   3,  80],
       [  0,   0,   0, ..., 449, 439, 296],
       ...,
       [  0,   0,   0, ...,  39,   1,  19],
       [  0,   0,   0, ..., 186,  31, 111],
       [  0,   0,   0, ...,   0,   1, 117]], dtype=int32)

In [ ]:
model = Sequential()
model.add(Embedding(500,120,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 49, 120)           60000     
                                                                 
 spatial_dropout1d (Spatial  (None, 49, 120)           0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 176)               209088    
                                                                 
 dense (Dense)               (None, 2)                 354       
                                                                 
Total params: 269442 (1.03 MB)
Trainable params: 269442 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
#Splitting the data into training and testing data

y = pd.get_dummies(df['Sentiment'])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state = 42)

In [ ]:
batch_size = 32
model.fit(X_train,y_train,epochs=1,batch_size=batch_size,verbose='auto')

18442/34533 [===============>..............] - ETA: 44:15 - loss: 0.5065 - accuracy: 0.7480